In [ ]:
!pip install openai # Colab에서 openai 라이브러리를 설치합니다.

import pandas as pd
import json
import time
import random
from tqdm import tqdm
import os
from openai import OpenAI # openai 라이브러리 import

# 1. 데이터셋 로드
file_path = 'canonical_jpop_dataset.csv'
try:
    df = pd.read_csv(file_path)
except FileNotFoundError:
    print(f"'{file_path}' 파일을 찾을 수 없습니다. Colab에 파일을 업로드했는지 확인해주세요.")
    data = {'artist': ['Artist A', 'Artist B'],
            'title': ['Title 1', 'Title 2'],
            'lyrics': ['사랑과 희망에 대한 노래입니다. 밝고 긍정적인 분위기.', '이별의 슬픔을 노래합니다. 어둡고 애절한 감성.']}
    df = pd.DataFrame(data)
    print("예시 데이터프레임으로 작업을 계속합니다.")

# --- 최종 실행 시 이 부분을 주석 처리하거나 df를 그대로 사용하세요 ---
test_df = df.sample(5)
print(f"--- 테스트 모드: 랜덤 5곡에 대해서만 작업을 수행합니다. ---")
# -----------------------------------------

# 2. OpenAI API 클라이언트 설정
try:
    from google.colab import userdata
    api_key = userdata.get('OPENAI_API_KEY')
    client = OpenAI(api_key=api_key)
except (ImportError, KeyError):
    print("Colab 비밀을 찾을 수 없습니다. API 키를 직접 입력해주세요.")
    api_key = "YOUR_API_KEY"
    if api_key == "YOUR_API_KEY":
        print("경고: 유효한 OpenAI API 키를 입력해야 합니다.")
        client = None
    else:
        client = OpenAI(api_key=api_key)

def post_process_summary(summary: str) -> str:
    """
    LLM이 생성한 요약문 중 특정 단어로 끝나는 경우에만 '~노래'로 교정하는 가벼운 후처리 함수.
    """
    summary = summary.strip().rstrip('.')
    endings_to_replace = ["가사", "이야기", "내용"]
    for ending in endings_to_replace:
        if summary.endswith(ending):
            return summary[:-len(ending)].strip() + " 노래"
    return summary

def get_features_from_openai(lyrics):
    """
    가사를 입력받아 OpenAI API를 통해 특징(태그, 요약)을 JSON 형태로 받아오는 함수.
    Few-shot 프롬프트를 사용하여 안정성을 높였습니다.
    """
    if not client:
        time.sleep(0.5)
        return {"tags": random.sample(["사랑", "희망", "청춘"], 3), "summary": "API 클라이언트가 설정되지 않아 생성된 임시 요약입니다."}

    # Few-shot 프롬프트: 좋은 예시 2개를 제공하여 모델의 결과물을 유도
    system_prompt = """
    너는 J-POP 가사를 분석하여 JSON 형식으로 결과를 반환하는 AI야.
    주어진 가사의 핵심 내용을 요약하고, 분위기에 맞는 태그 3개를 추출해야 해.
    아래 예시와 완벽하게 동일한 형식으로 응답해야 한다.
    """

    user_prompt = f"""
    [가사]:
    "夢ならばどれほどよかったでしょう
    未だにあなたのことを夢にみる..."
    (꿈이라면 얼마나 좋았을까 아직도 당신을 꿈에서 봐...)

    [결과]:
    {{
      "tags": ["이별", "슬픔", "상실감"],
      "summary": "이별한 상대를 잊지 못하고 꿈속에서까지 그리워하는 깊은 슬픔을 담은 노래"
    }}

    [가사]:
    "間違って傷つけて　変わってくことが怖かった
    でもいつだって僕ら　始まってる..."
    (실수하고 상처 입고 변해가는 것이 두려웠어 하지만 언제나 우리는 시작되고 있어...)

    [결과]:
    {{
      "tags": ["시작", "용기", "성장"],
      "summary": "실수를 두려워하지 않고 새로운 시작을 향해 나아가려는 용기와 성장에 대한 노래"
    }}

    [가사]:
    "{lyrics}"

    [결과]:
    """

    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo-0125",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                response_format={"type": "json_object"},
                temperature=0.5,
            )

            content = response.choices[0].message.content
            result = json.loads(content)

            if "tags" in result and "summary" in result:
                return result
            else:
                print(f"오류: 응답 JSON에 필수 키가 없습니다. 응답: {result}")
                return {"tags": ["파싱오류"], "summary": "필수 키 누락"}

        except Exception as e:
            print(f"API 요청 오류: {e}. {attempt + 1}/{max_retries} 번째 재시도 중...")
            time.sleep(2 ** attempt)

    return {"tags": ["API호출실패"], "summary": "API 호출 실패"}


# 3. 데이터셋에 특징(피처) 추가 작업 및 실시간 결과 출력
new_data = []
# --- 전체 실행 시 'test_df'를 'df'로 변경하세요 ---
for index, row in tqdm(test_df.iterrows(), total=test_df.shape[0], desc="특징 추출 중"):
    lyrics = row['lyrics']

    features = {}
    if not isinstance(lyrics, str) or len(lyrics.strip()) == 0:
        features = {"tags": ["가사없음"], "summary": "가사 없음"}
    else:
        # 1차 요청 (Few-shot으로 정확도 향상)
        features = get_features_from_openai(lyrics)

        # 가벼운 후처리로 문장 끝만 다듬기
        features['summary'] = post_process_summary(features.get('summary', ''))

    new_row = row.to_dict()
    new_row['tags'] = features.get('tags', [])
    new_row['summary'] = features.get('summary', 'N-A')
    if 'sentiment' in new_row:
        del new_row['sentiment']

    new_data.append(new_row)

    print(f"\n[결과] {row.get('artist', 'N/A')} - {row.get('title', 'N/A')}")
    print(f"  - 태그: {new_row['tags']}")
    print(f"  - 요약: {new_row['summary']}")
    print("-" * 25)


# 4. 결과 샘플링 검토는 생략
print("\n--- 모든 테스트 작업 완료 ---")


# 5. 최종 데이터셋을 JSON 파일로 저장
output_filename = 'jpop_dataset_with_features.json'
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(new_data, f, ensure_ascii=False, indent=4)

print(f"\n결과가 '{output_filename}' 파일로 저장되었습니다.")
""

--- 테스트 모드: 랜덤 5곡에 대해서만 작업을 수행합니다. ---


특징 추출 중:  20%|██        | 1/5 [00:01<00:04,  1.09s/it]


[결과] 嵐 (ARASHI) - Love so sweet
  - 태그: ['사랑', '이별', '추억']
  - 요약: 서로를 사랑하게 되어 빛나는 순간부터 이별의 아픔과 함께 떠나는 추억을 담은 노래
-------------------------


특징 추출 중:  40%|████      | 2/5 [00:01<00:02,  1.06it/s]


[결과] Hemenway(헤멘웨이) - 優しい言葉 (Yasashii Kotobai / 상냥한 말)
  - 태그: ['이별', '회상', '그리움']
  - 요약: 지나간 이별과 회상에 젖어있는 노래. 그리움과 회한을 담은 가사 속에서 향수를 느낄 수 있음
-------------------------


...

특징 추출 중:  80%|████████  | 4/5 [00:04<00:01,  1.09s/it]


[결과] ZUTOMAYO - MIRROR TUNE
  - 태그: ['도전', '자기계발', '긍정']
  - 요약: 도전과 성장을 향한 긍정적인 마음가짐을 담은 노래. 자기계발과 자신을 믿는 용기를 주제로 함
-------------------------


특징 추출 중: 100%|██████████| 5/5 [00:05<00:00,  1.02s/it]


[결과] Lisa Ono(리사 오노/小野リサ) - Dans Mon Ile
  - 태그: ['낭만', '낙원', '해변']
  - 요약: 해변가의 낙원 같은 섬에서 아무것도 하지 않고 햇빛을 쬐며 휴식을 취하는 모습을 담은 로맨틱한 노래
-------------------------

--- 모든 테스트 작업 완료 ---

결과가 'jpop_dataset_with_features.json' 파일로 저장되었습니다.


''

### 전체 데이터셋에 대해 수행

In [ ]:
!pip install openai # Colab에서 openai 라이브러리를 설치합니다.

import pandas as pd
import json
import time
import random
from tqdm import tqdm
import os
from openai import OpenAI # openai 라이브러리 import

# 1. 데이터셋 로드 및 손실 방지 로직
input_filename = 'canonical_jpop_dataset.csv'
output_filename = 'canonical_jpop_dataset_with_features.csv'

# 결과 파일이 이미 있는지 확인하여 이어하기 기능 구현
if os.path.exists(output_filename):
    print(f"'{output_filename}' 파일이 존재합니다. 중단된 지점부터 작업을 이어갑니다.")
    df = pd.read_csv(output_filename)
else:
    print(f"새로운 작업을 시작합니다. 원본 파일: '{input_filename}'")
    try:
        df = pd.read_csv(input_filename)
        # 새로운 작업을 시작할 때만 컬럼 추가
        df['tags'] = None
        df['summary'] = None
    except FileNotFoundError:
        print(f"'{input_filename}' 파일을 찾을 수 없습니다. Colab에 파일을 업로드했는지 확인해주세요.")
        # 예시 데이터프레임 생성
        data = {'artist': ['Artist A', 'Artist B'],
                'title': ['Title 1', 'Title 2'],
                'lyrics': ['사랑과 희망에 대한 노래입니다. 밝고 긍정적인 분위기.', '이별의 슬픔을 노래합니다. 어둡고 애절한 감성.']}
        df = pd.DataFrame(data)
        df['tags'] = None
        df['summary'] = None
        print("예시 데이터프레임으로 작업을 계속합니다.")

print(f"--- 전체 데이터셋({len(df)}곡)에 대한 작업을 시작합니다. ---")
# -----------------------------------------

# 2. OpenAI API 클라이언트 설정
try:
    from google.colab import userdata
    api_key = userdata.get('OPENAI_API_KEY')
    client = OpenAI(api_key=api_key)
except (ImportError, KeyError):
    print("Colab 비밀을 찾을 수 없습니다. API 키를 직접 입력해주세요.")
    api_key = "YOUR_API_KEY"
    if api_key == "YOUR_API_KEY":
        print("경고: 유효한 OpenAI API 키를 입력해야 합니다.")
        client = None
    else:
        client = OpenAI(api_key=api_key)

def post_process_summary(summary: str) -> str:
    """
    LLM이 생성한 요약문 중 특정 단어로 끝나는 경우에만 '~노래'로 교정하는 가벼운 후처리 함수.
    """
    summary = summary.strip().rstrip('.')
    endings_to_replace = ["가사", "이야기", "내용"]
    for ending in endings_to_replace:
        if summary.endswith(ending):
            return summary[:-len(ending)].strip() + " 노래"
    return summary

def get_features_from_openai(lyrics):
    """
    가사를 입력받아 OpenAI API를 통해 특징(태그, 요약)을 JSON 형태로 받아오는 함수.
    Few-shot 프롬프트를 사용하여 안정성을 높였습니다.
    """
    if not client:
        time.sleep(0.5)
        return {"tags": random.sample(["사랑", "희망", "청춘"], 3), "summary": "API 클라이언트가 설정되지 않아 생성된 임시 요약입니다."}

    # Few-shot 프롬프트: 좋은 예시 2개를 제공하여 모델의 결과물을 유도
    system_prompt = """
    너는 J-POP 가사를 분석하여 JSON 형식으로 결과를 반환하는 AI야.
    주어진 가사의 핵심 내용을 요약하고, 분위기에 맞는 태그 3개를 추출해야 해.
    아래 예시와 완벽하게 동일한 형식으로 응답해야 한다.
    """

    user_prompt = f"""
    [가사]:
    "夢ならばどれほどよかったでしょう
    未だにあなたのことを夢にみる..."
    (꿈이라면 얼마나 좋았을까 아직도 당신을 꿈에서 봐...)

    [결과]:
    {{
      "tags": ["이별", "슬픔", "상실감"],
      "summary": "이별한 상대를 잊지 못하고 꿈속에서까지 그리워하는 깊은 슬픔을 담은 노래"
    }}

    [가사]:
    "間違って傷つけて　変わってくことが怖かった
    でもいつだって僕ら　始まってる..."
    (실수하고 상처 입고 변해가는 것이 두려웠어 하지만 언제나 우리는 시작되고 있어...)

    [결과]:
    {{
      "tags": ["시작", "용기", "성장"],
      "summary": "실수를 두려워하지 않고 새로운 시작을 향해 나아가려는 용기와 성장에 대한 노래"
    }}

    [가사]:
    "{lyrics}"

    [결과]:
    """

    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo-0125",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                response_format={"type": "json_object"},
                temperature=0.5,
            )

            content = response.choices[0].message.content
            result = json.loads(content)

            if "tags" in result and "summary" in result:
                return result
            else:
                print(f"오류: 응답 JSON에 필수 키가 없습니다. 응답: {result}")
                return {"tags": ["파싱오류"], "summary": "필수 키 누락"}

        except Exception as e:
            print(f"API 요청 오류: {e}. {attempt + 1}/{max_retries} 번째 재시도 중...")
            time.sleep(2 ** attempt)

    return {"tags": ["API호출실패"], "summary": "API 호출 실패"}


# 3. 데이터셋에 특징(피처) 추가 작업
# --- 전체 데이터셋(df)에 대해 작업 수행 ---
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="특징 추출 중"):

    # --- 이어하기 로직 ---
    # 'summary' 컬럼에 유효한 값이 이미 있다면, 해당 곡은 건너뜁니다.
    if pd.notna(row['summary']) and row['summary'] != "" and row['summary'] is not None:
        continue
    # --------------------

    lyrics = row['lyrics']

    if not isinstance(lyrics, str) or len(lyrics.strip()) == 0:
        tags = ["가사없음"]
        summary = "가사 없음"
    else:
        features = get_features_from_openai(lyrics)
        tags = features.get('tags', [])
        summary = post_process_summary(features.get('summary', ''))

    df.loc[index, 'tags'] = str(tags)
    df.loc[index, 'summary'] = summary

    print(f"\n[결과] {row.get('artist', 'N/A')} - {row.get('title', 'N/A')}")
    print(f"  - 생성된 태그: {str(tags)}")
    print(f"  - 생성된 요약: {summary}")
    print("-" * 25)

    # --- 중간 저장 로직 ---
    # 50곡을 처리할 때마다 CSV 파일에 중간 결과를 저장합니다.
    if (index + 1) % 50 == 0:
        df.to_csv(output_filename, index=False, encoding='utf-8-sig')
        tqdm.write(f"\n[진행 상황 저장됨: {index + 1}/{len(df)} 완료]")
    # --------------------


# 4. 작업 완료 후 샘플링 검토
print("\n--- 작업 완료 후, 최종 샘플링 검토 ---")
random_samples = df.sample(min(5, len(df)))
for index, sample in random_samples.iterrows():
    print(f"\n[샘플 {index}]")
    print(f"  - 아티스트: {sample.get('artist', 'N/A')}")
    print(f"  - 제  목: {sample.get('title', 'N/A')}")
    print(f"  - 생성된 태그: {sample.get('tags', '[]')}")
    print(f"  - 생성된 요약: {sample.get('summary', 'N/A')}")


# 5. 최종 데이터셋을 새로운 CSV 파일로 저장
df.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"\n결과가 '{output_filename}' 파일로 저장되었습니다.")


새로운 작업을 시작합니다. 원본 파일: 'canonical_jpop_dataset.csv'
--- 전체 데이터셋(460곡)에 대한 작업을 시작합니다. ---


특징 추출 중:   0%|          | 1/460 [00:00<06:51,  1.12it/s]


[결과] Kenshi Yonezu(켄시 요네즈/米津 玄師) - Lemon
  - 생성된 태그: ['이별', '회상', '그리움']
  - 생성된 요약: 과거의 사랑과 이별에 대한 회상과 그리움을 담은 노래
-------------------------


특징 추출 중:   0%|          | 2/460 [00:01<06:53,  1.11it/s]


[결과] NiziU (니쥬) - Make you happy
  - 생성된 태그: ['사랑', '행복', '감사']
  - 생성된 요약: 상대방을 행복하게 하고 싶어하는 사랑과 감사의 마음을 담은 노래
-------------------------


...


[결과] NiziU - Blue Moon
  - 생성된 태그: ['사랑', '회상', '그리움']
  - 생성된 요약: 먼 거리에 있어도 늘 함께하는 마음으로 연결되어 있는 사랑의 감정을 회상하며 그리워하는 노래
-------------------------


특징 추출 중: 100%|█████████▉| 459/460 [08:53<00:01,  1.25s/it]


[결과] Special Favorite Music(스페셜 페이보릿 뮤직) - Dribble
  - 생성된 태그: ['사랑', '추억', '이별']
  - 생성된 요약: 사랑하는 이를 잊지 못하고 추억에 잠기는 가슴 아픈 이별의 아픔을 담은 노래
-------------------------


특징 추출 중: 100%|██████████| 460/460 [08:54<00:00,  1.16s/it]


[결과] TWICE - BETTER
  - 생성된 태그: ['사랑', '희망', '우정']
  - 생성된 요약: 서로의 꿈을 향해 함께 보내는 날들, 서로를 지지하고 힘을 주며 더 나은 미래를 향해 나아가는 사랑과 우정에 대한 노래
-------------------------

--- 작업 완료 후, 최종 샘플링 검토 ---

[샘플 268]
  - 아티스트: King Gnu(킹 누)
  - 제  목: BOY
  - 생성된 태그: ['꿈', '도전', '희망']
  - 생성된 요약: 자신의 꿈을 향해 도전하고 희망을 잃지 않는 강인한 의지를 담은 노래

[샘플 372]
  - 아티스트: Number_i
  - 제  목: FUJI
  - 생성된 태그: ['자유로움', '긍정적', '파워']
  - 생성된 요약: 자유롭고 긍정적인 에너지를 담은 노래. 자신을 믿고 꿈을 향해 나아가는 모험적인 노래

[샘플 370]
  - 아티스트: imase
  - 제  목: Shine Out
  - 생성된 태그: ['희망', '도전', '긍정']
  - 생성된 요약: 희망찬 미래를 향해 달려가는 도전과 긍정적인 마음가짐을 담은 노래

[샘플 24]
  - 아티스트: SEKAI NO OWARI(세카이노 오와리)
  - 제  목: RPG
  - 생성된 태그: ['희망', '우정', '도전']
  - 생성된 요약: 자신을 믿고 새로운 도전에 나서며 우정과 희망으로 가득한 노래

[샘플 215]
  - 아티스트: IZ*ONE
  - 제  목: 幻想童話 (Secret Story of the Swan)
  - 생성된 태그: ['로맨스', '꿈', '마법']
  - 생성된 요약: 로맨틱한 분위기 속에서 사랑하는 이와 함께하는 꿈 속의 여행을 표현한 가사, 마법 같은 순간을 꿈꾸며 함께하는 이들의 로맨틱한 노래

결과가 'canonical_jpop_dataset_with_features.csv' 파일로 저장되었습니다.


In [4]:
import pandas as pd
import json
from collections import Counter
from tqdm import tqdm

# --- 1. 최종 핵심 태그 목록 (직접 재분류 및 통합 완료) ---
# '고유_테마'를 '콘셉트'로 변경했습니다.
FINAL_MAJOR_TAGS = sorted([
    '갈등', '감동', '감사', '감성', '계절', '고난', '고독', '고백', '콘셉트',
    '그리움', '긍정', '꿈', '도시', '도전', '모험', '몽환적', '사랑', '사회비판',
    '성장', '소망', '슬픔', '우정', '운명', '욕망', '음악', '이별', '인내',
    '인생', '일상', '자아성찰', '자신감', '자유', '절망', '즐거움', '청춘',
    '추억', '춤', '파티', '행복', '혼란', '회상', '후회', '희망', '기타'
])

# --- 2. 수동 태그 매핑 (Minor -> Major) ---
# '고유_테마'로 매핑되던 태그들을 '콘셉트'로 변경했습니다.
TAG_MAPPING = {
    # 사랑/관계
    '로맨스': '사랑', '로맨틱': '사랑', '연애': '사랑', '연인': '사랑', '인연': '사랑',
    '재회': '사랑', '헌신': '사랑', '희생': '사랑', '인간관계': '우정', '소통': '우정',
    '친구': '우정', '신뢰': '우정', '지지': '우정', '가족': '사랑', '약속': '사랑',
    '복잡한 관계': '갈등', '엇갈림': '갈등', '오해': '갈등',

    # 감정 (긍정)
    '기쁨': '즐거움', '낙천': '긍정', '낙천적': '긍정', '밝은': '긍정', '신나는': '즐거움',
    '환희': '즐거움', '활기': '즐거움', '흥겨운': '즐거움', '흥분': '즐거움', '평온': '일상',
    '평화': '일상', '평화로움': '일상', '휴식': '일상', '축복': '감사', '축하': '즐거움',

    # 감정 (부정)
    '고민': '슬픔', '고통': '슬픔', '괴로움': '슬픔', '무력감': '절망', '분노': '갈등',
    '상실감': '슬픔', '상처': '슬픔', '아픔': '슬픔', '애잔': '슬픔', '외로움': '고독',
    '우울': '슬픔', '자책': '후회', '허전함': '슬픔', '회한': '후회', '피로': '일상',
    '스트레스': '일상', '괴로운 상상': '혼란', '복잡한 감정': '혼란',

    # 자아/성장
    '가치': '자아성찰', '내면': '자아성찰', '다짐': '도전', '도약': '성장', '도전정신': '도전',
    '독립': '성장', '변화': '성장', '사명감': '도전', '생각': '자아성찰', '선택': '인생',
    '성공': '성장', '승리': '성장', '용기': '자신감', '의미': '인생', '이해': '성장',
    '자기 성찰': '자아성찰', '자기 표현': '자아성찰', '자기애': '자신감', '자기인식': '자아성찰',
    '자아': '자아성찰', '자아 발견': '자아성찰', '자아 표현': '자아성찰', '자아인식': '자아성찰',
    '자아탐색': '자아성찰', '정체성': '자아성찰', '투지': '도전', '포기하지않음': '인내',

    # 추상/테마
    '갈망': '소망', '기억': '추억', '기적': '소망', '미래': '희망', '시간': '인생',
    '운명': '운명', '이상': '꿈', '현실': '인생', '환상': '몽환적', '환상적': '몽환적',

    # 고유/특수 테마 -> '콘셉트'로 통합
    '마법': '몽환적', '미스테리': '콘셉트', '미지': '모험', '별': '몽환적', '사이버펑크': '콘셉트',
    '사회문제': '사회비판', '사회비판': '사회비판', '상상': '꿈', '우주': '몽환적', '영화': '콘셉트',
    '전쟁': '갈등', '전투': '갈등', '싸움': '갈등', '탐험': '모험', '파라다이스': '몽환적',
    '낙원': '몽환적', '파멸': '절망', '자살': '절망',

    # 이벤트/사물
    '봄': '계절', '여름': '계절', '크리스마스': '계절', '축제': '파티', '음주': '파티',
    '해변': '계절', '패션': '일상', '팝': '음악', '아이돌': '음악',

    # 기타
    '가식': '갈등', '구원': '희망', '긍정적': '긍정', '도움': '우정', '소문': '혼란',
    '소중한 순간': '추억', '소중함': '추억', '유혹': '욕망', '잊지 못함': '그리움',
    '자기계발': '성장', '자기중심': '자아성찰', '자랑': '자신감', '젊음': '청춘',
    '창의성': '음악', '창작': '음악', '카롤': '기타', '특별한 순간': '추억', '파워': '자신감',
    '포기': '절망', '감각적': '감성', '매혹': '감성', '모던': '도시', '새로움': '도전',
    '시대를 초월한 사랑': '사랑'
}

# --- 3. 기본 설정 및 파일 경로 ---
INPUT_CSV = 'canonical_jpop_dataset_with_features.csv'
OUTPUT_CSV = 'canonical_jpop_dataset_final.csv'
OUTPUT_COLOR_MAP = 'tag_colors.json'

# --- 4. 데이터 로드 ---
print(f"'{INPUT_CSV}' 파일을 로드합니다...")
df = pd.read_csv(INPUT_CSV)

# --- 5. 정규화된 태그 생성 함수 ---
def normalize_tags(tags_str):
    try:
        # 1. 문자열을 리스트로 변환하고 기본 정리 (공백, 소문자)
        tags_list = [tag.strip().lower() for tag in eval(tags_str)]

        # 2. 수동 매핑을 기반으로 태그를 정규화
        normalized_list = [TAG_MAPPING.get(tag, tag) for tag in tags_list]

        # 3. 최종 핵심 태그 목록에 있는 것만 필터링 (안전장치)
        final_list = [tag for tag in normalized_list if tag in FINAL_MAJOR_TAGS]

        # 4. 중복 제거 후 문자열로 반환
        return str(list(dict.fromkeys(final_list)))
    except:
        return "[]"

# --- 6. 새로운 'tags_normalized' 컬럼 생성 ---
print("새로운 'tags_normalized' 컬럼을 생성합니다...")
df['tags_normalized'] = df['tags'].apply(normalize_tags)
print("태그 정규화 완료.")

# --- 7. [개선됨] 태그별 고정 색상 맵 생성 (의미 기반) ---
print("의미 기반의 고정 색상 맵을 생성합니다...")
color_map = {
    '갈등': '#9E9E9E', '감동': '#FFB74D', '감사': '#FFF176', '감성': '#BA68C8',
    '계절': '#81C784', '고난': '#795548', '고독': '#78909C', '고백': '#F06292',
    '콘셉트': '#00ACC1', '그리움': '#90A4AE', '긍정': '#FFD54F', '꿈': '#7E57C2',
    '도시': '#607D8B', '도전': '#FF7043', '모험': '#00897B', '몽환적': '#CE93D8',
    '사랑': '#EC407A', '사회비판': '#424242', '성장': '#66BB6A', '소망': '#AED581',
    '슬픔': '#64B5F6', '우정': '#42A5F5', '운명': '#8E24AA', '욕망': '#E53935',
    '음악': '#26A69A', '이별': '#A1887F', '인내': '#BDBDBD', '인생': '#37474F',
    '일상': '#FFF59D', '자아성찰': '#5C6BC0', '자신감': '#FFEE58', '자유': '#29B6F6',
    '절망': '#212121', '즐거움': '#FFCA28', '청춘': '#4FC3F7', '추억': '#D7CCC8',
    '춤': '#AD1457', '파티': '#F48FB1', '행복': '#FFC107', '혼란': '#BCAAA4',
    '회상': '#C5E1A5', '후회': '#90A4AE', '희망': '#FFA726', '기타': '#E0E0E0'
}

# --- 8. 결과물 저장 ---
print(f"'{OUTPUT_CSV}' 파일로 최종 데이터셋을 저장합니다...")
df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')

print(f"'{OUTPUT_COLOR_MAP}' 파일로 색상 맵을 저장합니다...")
with open(OUTPUT_COLOR_MAP, 'w', encoding='utf-8') as f:
    json.dump(color_map, f, ensure_ascii=False, indent=4)

print("\n--- 모든 전처리 작업 완료! ---")
print(f"'{OUTPUT_CSV}'와 '{OUTPUT_COLOR_MAP}' 파일이 성공적으로 생성되었습니다.")


'canonical_jpop_dataset_with_features.csv' 파일을 로드합니다...
새로운 'tags_normalized' 컬럼을 생성합니다...
태그 정규화 완료.
의미 기반의 고정 색상 맵을 생성합니다...
'canonical_jpop_dataset_final.csv' 파일로 최종 데이터셋을 저장합니다...
'tag_colors.json' 파일로 색상 맵을 저장합니다...

--- 모든 전처리 작업 완료! ---
'canonical_jpop_dataset_final.csv'와 'tag_colors.json' 파일이 성공적으로 생성되었습니다.
